In [1]:
import pandas as pd
import numpy as np

from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [2]:
avisos = pd.read_csv('Predecir/avisos_detalles_train.csv')
postulantes = pd.read_csv('Predecir/postulantes_datos_train.csv')
postulaciones = pd.read_csv('Predecir/df_postulaciones.csv')

In [3]:
del postulantes['Unnamed: 0']
del avisos['Unnamed: 0']
del postulaciones['Unnamed: 0']

In [4]:
avisos_si = avisos.copy()
avisos_si['se_postulo'] = 1

In [5]:
postulaciones = postulaciones.sample(frac=0.20)

In [6]:
postulaciones = pd.merge(postulaciones, avisos_si, how='inner', on=['idaviso'])
postulaciones = pd.merge(postulaciones, postulantes, how='inner', on=['idpostulante'])

In [7]:
def random_fill(row):
    return np.random.choice(avisos['idaviso'])

In [8]:
while (len(postulaciones[postulaciones["se_postulo"]==1])*0.5 > len(postulaciones[postulaciones["se_postulo"]==0])):
    relleno = postulantes.sample(frac=0.5).copy()
    relleno['idaviso'] = np.nan
    relleno['idaviso'] = relleno.apply(random_fill, axis=1)
    relleno = pd.merge(relleno, avisos, how='inner', on=['idaviso'])
    postulaciones = pd.merge(postulaciones, relleno, how='outer')
    postulaciones = postulaciones.fillna({'se_postulo' : 0})

In [9]:
postulaciones = postulaciones.dropna()

In [10]:
se_postulo = postulaciones['se_postulo']
del postulaciones['se_postulo']
del postulaciones['idaviso']
del postulaciones['idpostulante']
postulaciones.head()

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,incorp_inmediata,...,total_vistas,vistas_median,cant_dias,nombre,estado,sexo,edad,vistas_postulante,cant_dias_actividad,postu_postulante
0,1,1,24,40,0,0,0,0,0,0,...,160.0,1.0,27.0,1.0,2.0,2,40.0,23.0,56.0,23.0
1,1,1,1,556,0,0,0,0,0,0,...,96.0,1.0,58.0,1.0,2.0,2,40.0,23.0,56.0,23.0
3,1,1,52,27,0,0,0,0,1,0,...,128.0,1.0,79.0,1.0,2.0,2,40.0,23.0,56.0,23.0
4,1,1,8,35,0,0,0,0,0,0,...,27.0,1.0,29.0,1.0,2.0,2,40.0,23.0,56.0,23.0
5,1,1,24,40,0,0,0,0,0,0,...,160.0,1.0,27.0,1.0,2.0,1,45.0,7.0,26.0,7.0


In [11]:
X_train, X_test, y_train, y_test = train_test_split(postulaciones, se_postulo, test_size=0.15, random_state=7)

In [12]:
seed = 7
kfold = model_selection.KFold(n_splits=5, random_state=seed)
cart = DecisionTreeClassifier()
num_trees = 10
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold)
print(results.mean())

0.8524627958178561


In [13]:
model.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=7, verbose=0, warm_start=False)

In [14]:
to_predict  = pd.read_csv('Predecir/test_final_100k.csv')
to_predict  = pd.merge(to_predict, avisos, on = "idaviso",how="left")
to_predict = pd.merge(to_predict, postulantes, on = "idpostulante",how="left")
to_predict = to_predict.drop_duplicates(["idaviso","idpostulante"])

In [15]:
to_predict.shape

(100000, 46)

In [16]:
del to_predict["idaviso"]
del to_predict["idpostulante"]
to_predict = to_predict.dropna()

In [17]:
to_sumit = model.predict(to_predict.drop("id",axis = "columns"))

In [18]:
to_predict["prediccion"] = to_sumit

In [19]:
res  =  pd.read_csv('Predecir/test_final_100k.csv')

res = pd.merge(res, avisos, on = "idaviso",how="left")
res = pd.merge(res, postulantes, on = "idpostulante",how="left")
res = res.drop_duplicates(["idaviso","idpostulante"])

In [20]:
import random

In [21]:
to_sumit = pd.merge(res,to_predict,on="id",how="left")["prediccion"]
to_sumit = to_sumit.fillna(random.getrandbits(1))
df = pd.DataFrame()
df["sepostulo"] = to_sumit
df["id"] = pd.read_csv('Predecir/test_final_100k.csv')["id"]
final = df.set_index("id")
final.shape

(100000, 1)

In [22]:
final.to_csv("final.csv")